# Demo of collaborative target tracking 

In [ ]:
import collaborative_bearing_estimation as cbe
import numpy as np

# %matplotlib notebook
%matplotlib Qt5
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse
import datetime

## setting up the scenario


In [ ]:

seed = np.random.randint(0,200)
print (seed)


#seed = 112 is not great for CCE vs CI possibly due to no implementation of rejecting inconsistent stuff
# seed= 67 # is a good case 
# seed= 62 # great for CCE but CI performs worse than KF possibly due to no rejection of inconsistent stuff
# seed = 197 # not great for CCE vs CI and almost matching Kalman
seed = 168 # this is a good case for CCE
# seed = 197 # also a very good case for CCE
# seed = 1 # also a good case
# good seeds 198, 106, 170, 

np.random.seed (seed)

#initialize figure
x_range_plot = np.array([[-25],[60]])
y_range_plot = np.array([[-70],[20]])









# agent 1 and 2 list of positions with initial position added (same starting position for both)
x1 = np.array([[-15],[0]])
x2 = np.array([[8],[15]])

# target
x = np.array([[10],[-12]])



s1 = .3 # forward speed
a1 = np.deg2rad(0) #steering angle
xh1 = np.array([[2],[-1]])

e1 = 6**2
P1 = e1 * np.eye(2)
P1_inv = 1/e1 *np.eye(2)

min_est_r1 = 2 # min est range
max_est_r1 = 70 # max est range
est_ang_e1 = np.deg2rad(12) # est angle error std 

s2 = .3
a2 = np.deg2rad(-90)
xh2 = xh1 #np.array([[0],[0]])

e2 = e1 # 100
P2 = e2 * np.eye(2)
P2_inv = 1/e2 *np.eye(2)

min_est_r2 = 2 # min est range
max_est_r2 = 70 # max est range
est_ang_e2 = np.deg2rad(10) # est angle error std 

dt = 1
T = 300 # simulation time

F_m = 1 # Frequency of measurements max =1 
F_c = 1 # Frequency of fusion max =1 

#create two agents that estimate the target
a_l_NC = []
a_l_C = []

agent1 = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, "CI")

a1_NC = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, None)
a1_CCE = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, "CCE")
a1_CI = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, "CI")
a1_Kalman = cbe.Observer(1, x1, s1, a1, min_est_r1 , max_est_r1, est_ang_e1, xh1, P1, P1_inv, "Kalman")

a_l_NC.append(a1_NC)
a_l_C.append(a1_CCE)
a_l_C.append(a1_CI)
a_l_C.append(a1_Kalman)

agent2 = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, "CCE") 
a2_NC = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, None)
a2_CCE = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, "CCE")
a2_CI = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, "CI")
a2_Kalman = cbe.Observer(2, x2, s2, a2, min_est_r2, max_est_r2, est_ang_e2, xh2, P2, P2_inv, "Kalman")

a_l_NC.append(a2_NC)
a_l_C.append(a2_CCE)
a_l_C.append(a2_CI)
a_l_C.append(a2_Kalman)





## simulation run

In [ ]:
e_1 = np.random.normal(np.deg2rad(0), a_l_NC[0].th_e, size= int(T/dt)) # agent 1's entire measurement error
e_2 = np.random.normal(0, a_l_NC[1].th_e, size= int(T/dt)) 

# fig = plt.figure()
# ax = fig.add_subplot(111, aspect='equal')
# ax.set_xlim(x_range_plot[0,0], x_range_plot[1,0])
# ax.set_ylim(y_range_plot[0,0], y_range_plot[1,0])
# plt.title("Measurement Ellipses")

# ax.scatter(x[0,0], x[1,0], s = 100, marker="*", alpha=.5, c = "c", label= "Target") 
# ax.scatter(x1[0,0], x1[1,0], s = 100, marker="o", alpha=.5, c = "r", label= "Agent_1") 
# ax.scatter(x2[0,0], x2[1,0], s = 100, marker="o", alpha=.5, c = "b", label= "Agent_2") 
# for t in range (4):
    
#     cm, Pm, Pm_inv, f = a1_NC.sense(x, e_1[t])
#     a1_NC.plot_meas(cm, Pm, ax)
    
#     cm, Pm, Pm_inv, f = a2_NC.sense(x, e_2[t])
#     a2_NC.plot_meas(cm, Pm, ax)
    
# ax.legend()

# fig = plt.figure()
# ax = fig.add_subplot(111, aspect='equal')
# ax.set_xlim(x_range_plot[0,0], x_range_plot[1,0])
# ax.set_ylim(y_range_plot[0,0], y_range_plot[1,0])
# plt.title("initial setup")
# ax.scatter(x[0,0], x[1,0], s = 200, marker="*", alpha=1, c = "g", label= "Target") 
# for a in a_l_NC:
    
#     a.plot_pos(ax)
    
#     if a.ID ==1:
            
#             cm, Pm, Pm_inv, f = a.sense(x, e_1[0])
            
#     else: 
#             cm, Pm, Pm_inv, f = a.sense(x, e_2[0])
        
        
#     a.plot_meas(cm, Pm, ax)    

In [ ]:





#     a.estimate(cm, Pm, Pm_inv, f)
#     a.plot_est(ax)
    
    

fig = plt.figure()
ax2 = fig.add_subplot(111, aspect='equal')
ax2.set_xlim(x_range_plot[0,0], x_range_plot[1,0])
ax2.set_ylim(y_range_plot[0,0], y_range_plot[1,0])
plt.title("Scenario Configuration", fontsize=12)
plt.xlabel("x (m)", fontsize=12)
plt.ylabel("y (m)", fontsize=12)

for a in a_l_C + a_l_NC :
    a.est_err(x) # capture the first estimation error
    

for a in a_l_NC:
    a.plot_pos(ax2, True)
    
    
for t in range (int(T/dt)):
    
    print ("simulation time = ", t)
#     for a in a_l_NC:
#             a.plot_pos(ax)
    
    if t % (1/F_m) ==0:        
        for a in a_l_NC + a_l_C:


            if a.ID ==1:

                cm, Pm, Pm_inv, f = a.sense(x, e_1[t])
                ax2.scatter(cm[0,0], cm[1,0], marker="o", alpha=.01, c = "r")
                a.plot_meas(cm, Pm, ax2)

            else: 
                cm, Pm, Pm_inv, f = a.sense(x, e_2[t])
                ax2.scatter(cm[0,0], cm[1,0], marker="o", alpha=.01, c = "b")
                a.plot_meas(cm, Pm, ax2)



            a.estimate(cm, Pm, Pm_inv, f)
        
    
    for a in a_l_C + a_l_NC : # after everyone has sensed then we can allow information exchange and move before the next round
        
        if t % (1/F_c) ==0:
            for m in [k for k in a_l_C if k.ID != a.ID and k.fuse_method == a.fuse_method ]: #only looking in collaborative agents with matching method
            
            
                print ("fusing agent {0}{1} with agent{2}{3}".format(a.ID,a.fuse_method, m.ID, m.fuse_method))
            
                a.fuse (m)
        
#         a.move(dt)
        a.est_err(x)
#         a.propagate(t) # deflates the confidence in prior initially less and eventually more before estimation
        
        
#         plt.pause(.8)
#         plt.show()
        
        
# for a in a_l_C + a_l_NC:
    
#     a.plot_est(ax)
ax2.scatter(x[0,0], x[1,0], s = 300, marker="*", alpha=1, c = "g", label= "Target")      


    
agent1.plot_est(ax2, "Agent 1 - Initial Estimate")
agent2.plot_est(ax2, "Agent 2 - Initial Estimate")


    
ax2.legend(fontsize=12)  
plt.show()

In [ ]:

plt.rcParams['text.usetex'] = True

fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
plt.title ( r'\textbf{Estimation Error}', fontsize=14)
plt.xlabel(r'\textbf{Time (s)}', fontsize=14)
plt.ylabel (r'$\Vert \hat{x}_k - p\Vert$ (m)', fontsize=14)

fig3 = plt.figure()
ax3 = fig3.add_subplot(111)
plt.title ( r'\textbf{Estimation Covariance}', fontsize=12)
plt.xlabel(r'\textbf{Time (s)}', fontsize=12)
plt.ylabel(r'$\det (\hat{P}_k)$', fontsize=12)

for a in a_l_C + a_l_NC:
    if a.fuse_method == "Kalman":
        color = 'g'
    if a.fuse_method == "CCE":
        color = 'blue'
    if a.fuse_method == "CI":
        color = "r"
    if a.fuse_method == None:
        color ='k'
        
    if a.ID ==1:
        ls = "-"
    else:
        ls = "--"

    ax2.plot(a.err_l, color=color, ls=ls, label="Agent_{0}, Method_{1}".format(a.ID, a.fuse_method))
    ax3.plot(a.conf_l, color=color, ls=ls, label="Agent_{0}, Method_{1}".format(a.ID, a.fuse_method))
    
ax2.legend(fontsize=14)
ax3.legend(fontsize=14)
plt.show()


In [ ]:
print (np.rad2deg(e_1[0]), np.rad2deg(e_2[0]))